In [ ]:
import praw
reddit = praw.Reddit(
    client_id="gRHEFX9H4lWzqR3HTNI-xg",
    client_secret="QFt-7pifzSBA1eNkG6NLnuK7zOZJrQ",
    user_agent="Assignment1_CityU",
)

In [ ]:
import pandas as pd

# Create empty lists to store data
combined_data = []

posts = reddit.subreddit('elonmusk').top(limit=None)

# Collect data from posts
for post in posts:
    post_data = {
        'Title': post.title,
        'Post ID': post.id,
        'Author': str(post.author),
        'URL': post.url,
        'Score': post.score,
        'Comment Count': post.num_comments,
        'Created': post.created_utc,
    }
    combined_data.append(post_data)

    # Collect data from comments for the current post
    for comment in post.comments.list():
        if isinstance(comment, praw.models.Comment):
            comment_data = {
                'Title': post.title,  # Use the post's title for the comment
                'Post ID': post.id,  # Use the post's ID for the comment
                'Author': str(comment.author),
                'URL': post.url,  # Use the post's URL for the comment
                'Score': comment.score,
                'Comment Count': 0,  # Comments on comments are not tracked
                'Created': comment.created_utc,
                'Comment Body': comment.body,
                'Comment Author': str(comment.author),
            }
            combined_data.append(comment_data)

# Create a DataFrame from the collected data
df = pd.DataFrame(combined_data)

In [ ]:
df

In [ ]:
df.to_csv('reddit_data.csv', index=False)

In [ ]:
! pip install langdetect

In [ ]:
import langdetect # Import the language detection function

# Load the existing DataFrame from a CSV file
df = pd.read_csv('reddit_data.csv')
# Create a function to detect the language of a text
def detect_language(text):
    try:
        return langdetect.detect(text)
    except:
        return 'unknown'  # Handle cases where language detection fails

# Apply the language detection function to the 'Title' and 'Comment Body' columns
df['Title Language'] = df['Title'].apply(detect_language)
df['Comment Language'] = df['Comment Body'].apply(detect_language)

# Filter only English posts and comments
english_df = df[(df['Title Language'] == 'en') & (df['Comment Language'] == 'en')]

# Drop the language detection columns
english_df.drop(['Title Language', 'Comment Language'], axis=1, inplace=True)

# Save the DataFrame with English posts and comments to a new CSV file
english_df.to_csv('english_reddit_data.csv', index=False)

In [31]:
import pandas as pd
# Load the DataFrame with English posts and comments
english_df = pd.read_csv('english_reddit_data.csv')

# Convert the 'Created' column to a datetime format
english_df['Created'] = pd.to_datetime(english_df['Created'], unit='s')
english_df['Created'] = pd.to_datetime(english_df['Created'], format='%Y-%m-%d %H:%M:%S')

In [32]:
english_df

,Title,Post ID,Author,URL,Score,Comment Count,Created,Comment Body,Comment Author
0,Elon Musk Leaves Presidential Councils,6epd1s,Litterball,https://twitter.com/elonmusk/status/8703699158...,9711,0,2017-06-01 20:21:23,There is really no point in staying on as an a...,Litterball
1,Elon Musk Leaves Presidential Councils,6epd1s,suredoit,https://twitter.com/elonmusk/status/8703699158...,575,0,2017-06-01 20:34:32,I am surprised this isnt on /r/Futurology,suredoit
2,Elon Musk Leaves Presidential Councils,6epd1s,RiseoftheTrumpwaffen,https://twitter.com/elonmusk/status/8703699158...,499,0,2017-06-01 20:31:45,I guess all that 'the only way to deal with Tr...,RiseoftheTrumpwaffen
3,Elon Musk Leaves Presidential Councils,6epd1s,mirrorsaw,https://twitter.com/elonmusk/status/8703699158...,906,0,2017-06-01 20:10:56,Was there any doubt this man would stick to hi...,mirrorsaw
4,Elon Musk Leaves Presidential Councils,6epd1s,jonb3838,https://twitter.com/elonmusk/status/8703699158...,34,0,2017-06-01 23:52:08,Every President wants to leave a stamp on Amer...,jonb3838
...,...,...,...,...,...,...,...,...,...
50969,Opinionated politician opens her mouth and get...,u4dltu,NaN,https://i.redd.it/atra15lxcqt81.png,1,0,2022-04-16 19:09:21,Well I didn't say it isn't getting better. I a...,NaN
50970,Opinionated politician opens her mouth and get...,u4dltu,NaN,https://i.redd.it/atra15lxcqt81.png,1,0,2022-04-16 19:12:24,I'm not sure that is true unless there is a pe...,NaN
50971,Opinionated politician opens her mouth and get...,u4dltu,NaN,https://i.redd.it/atra15lxcqt81.png,1,0,2022-04-16 19:18:50,I don't think we can have that evidence with c...,NaN
50972,Opinionated politician opens her mouth and get...,u4dltu,mrprogrampro,https://i.redd.it/atra15lxcqt81.png,1,0,2022-04-16 11:12:36,"Well, to be fair, I saw NYT and Time articles ...",mrprogrampro


In [33]:
# Create a new 'Date' column with the desired format '%Y-%m-%d'
english_df['Date'] = english_df['Created'].dt.strftime('%Y-%m-%d')

In [34]:
# Extract the minimum and maximum dates
min_date = english_df['Date'].min()
max_date = english_df['Date'].max()

print(f"Minimum Date: {min_date}")
print(f"Maximum Date: {max_date}")

Minimum Date: 2016-11-09
Maximum Date: 2023-10-08


In [22]:
from alpha_vantage.timeseries import TimeSeries

# Replace 'YOUR_API_KEY' with your AlphaVantage API key
api_key = 'J2OJF6SPJGM94TKD'

# Create an instance of the TimeSeries class
ts = TimeSeries(key=api_key, output_format='pandas')

symbol = 'TSLA'  # Tesla stock symbol


# Retrieve historical stock price data from AlphaVantage
stock_data, meta_data = ts.get_daily(symbol=symbol, outputsize='full')

# Convert the date index to a 'Date' column in the DataFrame
stock_data.reset_index(inplace=True)
stock_data.to_csv('stock_data.csv', index=False)

TypeError: TimeSeries.get_daily() got an unexpected keyword argument 'start'

In [52]:
# Load your AlphaVantage DataFrame
alpha_df = pd.read_csv('stock_data.csv')

# Convert the 'Date' column in the AlphaVantage DataFrame to datetime
alpha_df['date'] = pd.to_datetime(alpha_df['date'], format='%Y-%m-%d')
alpha_df['date'] = alpha_df['date'].dt.strftime('%Y-%m-%d')

# Slice the 'alpha_df' DataFrame to select rows within the date range in reddit data
alpha_sliced_df = alpha_df.loc[(alpha_df['date'] >= min_date) & (alpha_df['date'] <= max_date)]

# Reset the index if needed
alpha_sliced_df.reset_index(drop=True, inplace=True)

alpha_sliced_df = alpha_sliced_df.rename(columns={'date': 'Date'})


In [53]:
alpha_sliced_df

,Date,1. open,2. high,3. low,4. close,5. volume
0,2023-10-06,253.980,261.6500,250.65,260.53,118121812.0
1,2023-10-05,260.000,263.6000,256.25,260.05,119159214.0
2,2023-10-04,248.140,261.8600,247.60,261.16,129721567.0
3,2023-10-03,248.610,250.0200,244.45,246.53,101985305.0
4,2023-10-02,244.810,254.2799,242.62,251.60,123810402.0
...,...,...,...,...,...,...
1733,2016-11-15,182.780,186.4300,182.05,183.77,3902018.0
1734,2016-11-14,188.000,188.2500,178.19,181.45,6552205.0
1735,2016-11-11,184.240,188.8800,183.00,188.56,3988504.0
1736,2016-11-10,191.050,191.6100,180.42,185.35,6750341.0


In [64]:
# Merge the DataFrames based on the 'Date' column
merged_df = pd.merge(english_df,alpha_sliced_df, on='Date', how='inner')
# Save the merged data to a CSV file or perform further analysis
merged_df.to_csv('merged_data.csv', index=False)

In [65]:
merged_df

,Title,Post ID,Author,URL,Score,Comment Count,Created,Comment Body,Comment Author,Date,1. open,2. high,3. low,4. close,5. volume
0,Elon Musk Leaves Presidential Councils,6epd1s,Litterball,https://twitter.com/elonmusk/status/8703699158...,9711,0,2017-06-01 20:21:23,There is really no point in staying on as an a...,Litterball,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
1,Elon Musk Leaves Presidential Councils,6epd1s,suredoit,https://twitter.com/elonmusk/status/8703699158...,575,0,2017-06-01 20:34:32,I am surprised this isnt on /r/Futurology,suredoit,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
2,Elon Musk Leaves Presidential Councils,6epd1s,RiseoftheTrumpwaffen,https://twitter.com/elonmusk/status/8703699158...,499,0,2017-06-01 20:31:45,I guess all that 'the only way to deal with Tr...,RiseoftheTrumpwaffen,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
3,Elon Musk Leaves Presidential Councils,6epd1s,mirrorsaw,https://twitter.com/elonmusk/status/8703699158...,906,0,2017-06-01 20:10:56,Was there any doubt this man would stick to hi...,mirrorsaw,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
4,Elon Musk Leaves Presidential Councils,6epd1s,jonb3838,https://twitter.com/elonmusk/status/8703699158...,34,0,2017-06-01 23:52:08,Every President wants to leave a stamp on Amer...,jonb3838,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36825,Farm livin’ is the life for me,bk3xg3,Whatsittoyousmartguy,https://i.redd.it/d4ls5jno1xv21.jpg,7,0,2019-05-03 10:22:53,"well, is it?",Whatsittoyousmartguy,2019-05-03,243.86,256.61,243.49,255.03,23706771.0
36826,Farm livin’ is the life for me,bk3xg3,N8Disk,https://i.redd.it/d4ls5jno1xv21.jpg,6,0,2019-05-03 12:23:31,Verifying what you read on the internet is a g...,N8Disk,2019-05-03,243.86,256.61,243.49,255.03,23706771.0
36827,Farm livin’ is the life for me,bk3xg3,Apatomoose,https://i.redd.it/d4ls5jno1xv21.jpg,5,0,2019-05-03 12:47:58,That's because he takes suggestions and runs w...,Apatomoose,2019-05-03,243.86,256.61,243.49,255.03,23706771.0
36828,Farm livin’ is the life for me,bk3xg3,bradopolis,https://i.redd.it/d4ls5jno1xv21.jpg,5,0,2019-05-03 14:34:39,Is this real?,bradopolis,2019-05-03,243.86,256.61,243.49,255.03,23706771.0


In [66]:
import nltk
from nltk.stem import PorterStemmer

# Separate text columns from numerical columns
text_columns = ["Title", "Comment Body"]

# Initialize NLTK's Porter Stemmer
stemmer = PorterStemmer()

# Define a function for text preprocessing (stemming and normalization)
def preprocess_text(text):
    # Tokenize the text (split it into words)
    words = nltk.word_tokenize(text)

    # Apply stemming to each word
    stemmed_words = [stemmer.stem(word) for word in words]

    # Join the stemmed words back into a single string
    normalized_text = ' '.join(stemmed_words)

    return normalized_text

# Apply the preprocessing function to each text column
for col in text_columns:
    merged_df[col] = merged_df[col].apply(preprocess_text)


In [67]:
merged_df

,Title,Post ID,Author,URL,Score,Comment Count,Created,Comment Body,Comment Author,Date,1. open,2. high,3. low,4. close,5. volume
0,elon musk leav presidenti council,6epd1s,Litterball,https://twitter.com/elonmusk/status/8703699158...,9711,0,2017-06-01 20:21:23,there is realli no point in stay on as an advi...,Litterball,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
1,elon musk leav presidenti council,6epd1s,suredoit,https://twitter.com/elonmusk/status/8703699158...,575,0,2017-06-01 20:34:32,i am surpris thi isnt on /r/futurolog,suredoit,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
2,elon musk leav presidenti council,6epd1s,RiseoftheTrumpwaffen,https://twitter.com/elonmusk/status/8703699158...,499,0,2017-06-01 20:31:45,i guess all that 'the onli way to deal with tr...,RiseoftheTrumpwaffen,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
3,elon musk leav presidenti council,6epd1s,mirrorsaw,https://twitter.com/elonmusk/status/8703699158...,906,0,2017-06-01 20:10:56,wa there ani doubt thi man would stick to hi w...,mirrorsaw,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
4,elon musk leav presidenti council,6epd1s,jonb3838,https://twitter.com/elonmusk/status/8703699158...,34,0,2017-06-01 23:52:08,everi presid want to leav a stamp on america ....,jonb3838,2017-06-01,344.00,344.88,337.29,340.37,7580447.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36825,farm livin ’ is the life for me,bk3xg3,Whatsittoyousmartguy,https://i.redd.it/d4ls5jno1xv21.jpg,7,0,2019-05-03 10:22:53,"well , is it ?",Whatsittoyousmartguy,2019-05-03,243.86,256.61,243.49,255.03,23706771.0
36826,farm livin ’ is the life for me,bk3xg3,N8Disk,https://i.redd.it/d4ls5jno1xv21.jpg,6,0,2019-05-03 12:23:31,verifi what you read on the internet is a good...,N8Disk,2019-05-03,243.86,256.61,243.49,255.03,23706771.0
36827,farm livin ’ is the life for me,bk3xg3,Apatomoose,https://i.redd.it/d4ls5jno1xv21.jpg,5,0,2019-05-03 12:47:58,that 's becaus he take suggest and run with th...,Apatomoose,2019-05-03,243.86,256.61,243.49,255.03,23706771.0
36828,farm livin ’ is the life for me,bk3xg3,bradopolis,https://i.redd.it/d4ls5jno1xv21.jpg,5,0,2019-05-03 14:34:39,is thi real ?,bradopolis,2019-05-03,243.86,256.61,243.49,255.03,23706771.0


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Combine the preprocessed text columns into a single column
merged_df['Combined_Text'] = merged_df['Title'] + ' ' + merged_df['Comment Body']

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the combined text data
tfidf_matrix = tfidf_vectorizer.fit_transform(merged_df['Combined_Text'])

# Create a DataFrame from the TF-IDF matrix
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Print the TF-IDF DataFrame
print(tfidf_df)


        00  000  0000  00001  0000127445  0000127476  0000128219  00001c  \
0      0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   
1      0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   
2      0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   
3      0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   
4      0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   
...    ...  ...   ...    ...         ...         ...         ...     ...   
36825  0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   
36826  0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   
36827  0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   
36828  0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   
36829  0.0  0.0   0.0    0.0         0.0         0.0         0.0     0.0   

       0001  000119312518035345  ...  ˈkansəl  ˈkämənˌtāt   δv   πi  \
0       0.0     